# Introduction

## Q1. Running Elastic

In [1]:
!curl localhost:9200

{
  "name" : "3f3a5d0e0104",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "TkoHtD8BSo6hoTWJMMd6vA",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


`version.build_hash` value is dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

## Getting the data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [3]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 544.39it/s]


## Q3. Searching

In [35]:
query = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][0]['_score']

44.50556

## Q4. Filtering

In [36]:
query = "How do copy a file to a Docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][-1]["_source"]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [39]:
context_list = []
for hit in response['hits']['hits']:
    question = hit["_source"]["question"]
    text = hit["_source"]["text"]
    context_list.append(f"""
Q: {question}
A: {text}
""".strip())

In [42]:
question = "How do copy a file to a Docker container?"
context = "\n\n".join(context_list)
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [44]:
len(prompt_template)

1446

## Q6. Tokens

In [45]:
import tiktoken

In [46]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [49]:
len(encoding.encode(prompt_template))

320

In [50]:
encoding.decode_single_token_bytes(63842)

b"You're"

## Bonus: generating the answer

In [64]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv();

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

In [68]:
prompt_template

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

In [65]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt_template
)

In [67]:
response.text

'To copy a file to a Docker container, use the `docker cp` command. The syntax is: `docker cp /path/to/local/file_or_directory container_id:/path/in/container`. You can also provide the folder containing the files that you want to copy over in the Dockerfile using the command: `COPY ["src/predict.py", "models/xgb_model.bin", "./"]`.\n'

## Bonus: calculating the costs

In [72]:
input = 150
output = 250
requests_count = 1000
input_per_1000 = 0.005
output_per_1000 = 0.015
costs = (input * input_per_1000) + (output * output_per_1000)
'$' + str(costs)

'$4.5'